# Prepare

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys,psutil
from collections import Counter


In [2]:
import os
import json
import cv2
from matplotlib import pyplot as plt

# 找出所有json文件路径，保存下来

In [4]:
json_fp = "/home/zhoutong/facedata/all_json_file_path.txt"

In [19]:
# 找出所有json文件路径，保存下来
def write2file_useful_json(output_fp="/home/zhoutong/facedata/all_json_file_path.txt"):
    base_dir = "/home/zhoutong/facedata/CASIA-maxpy-clean"
    sub_dir_list = [os.path.join(base_dir, i) for i in os.listdir(base_dir)]
    res_path_list = []
    for sub_dir in sub_dir_list:
        if os.path.isdir(sub_dir):
            res_path_list.extend([os.path.join(sub_dir, i) for i in os.listdir(sub_dir) if ".json" in i])
    with open(output_fp, "w") as f:
        for p in res_path_list:
            f.writelines(p + "\n")
    return None

In [ ]:
# 找出所有json文件路径，保存下来
write2file_useful_json(json_fp)

# fp_list 存有所有json文件路径

In [5]:
with open(json_fp, "r") as f:
    fp_list = [i.strip() for i in f.readlines()]

# 逐一打开json文件，按描述取出人脸图像

In [135]:
def process_img(img,rect_list,expand_r=0.15):
    img_list = []
    H,W,_ = img.shape
    for rect in rect_list:
        (top,left,width,height) = (rect['top'],rect['left'],rect['width'],rect['height'])
        top = int(top - expand_r*height)
        height = int((1+expand_r*2)*height)
        left = int(left - expand_r*width)
        width = int((1+expand_r*2)*width)
        top = top if top>0 else 0
        left = left if left >0 else 0
        img_list.append(img[top:top+height,left:left+width])
    return img_list


In [ ]:
for fp in tqdm_notebook(fp_list[:]):
    new_fp = get_new_fp(fp)
    img_fp = os.path.splitext(json_fp)[0]+".jpg"
    img = cv2.imread(img_fp)
    with open(json_fp,"r") as f:
        json_info = json.load(f)
    rect_list = [json_info['face_rectangle'] for json_info in content['faces']]


In [ ]:
def process_img_old(img,rect_list):
    img_list = []
    for rect in rect_list:
        (top,left,width,height) = (rect['top'],rect['left'],rect['width'],rect['height'])
        img_list.append(img[top:top+height,left:left+width])
    return img_list

def prepare(json_fp):
    img_fp = os.path.splitext(json_fp)[0]+".jpg"
    with open(json_fp,"r") as f:
        content = json.load(f)
    img = cv2.imread(img_fp)
    rect_list = [i['face_rectangle'] for i in content['faces']]
    attr_list = [i['attributes']['ethnicity']['value'] for i in content['faces']]
    face_list = process_img(img,rect_list)
    return face_list,attr_list

def get_new_fp(fp="/home/zhoutong/facedata/CASIA-maxpy-clean/0000133/007.json"):
    CASIA_dir,img_collection_dir = os.path.split(os.path.dirname(fp))
    f_name = img_collection_dir+"_"+os.path.basename(fp).replace("json","jpg")
    f_dir = os.path.abspath(os.path.join(os.path.join(CASIA_dir,"../"),"prepared_data","face_img"))
    return os.path.join(f_dir,f_name)

def show(face,eth):
    _ = plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
    print("ethnicity: ",eth)
    plt.show()

In [139]:
for fp in tqdm_notebook(fp_list[0:]):
    new_fp = get_new_fp(fp)
#     print(fp)
#     print(new_fp)
    face_list,ethnicity_list = prepare(fp)
    with open("/home/zhoutong/facedata/prepared_data/enthnicity.csv","a") as f:
        for face,eth in zip(face_list,ethnicity_list):
    #         show(face,eth)
            save = cv2.imwrite(new_fp,face)
            if not save:
                print("Fail: [fp]:{} [new_fp]:{}".format(fp,new_fp))
            else:
                f.writelines(new_fp+"\t"+eth+"\n")

In [160]:
with open("/home/zhoutong/facedata/prepared_data/enthnicity.csv","r") as f:
    content = [i.strip().split("\t") for i in f.readlines()][:10]


# 【题外操作】人种数据分目录存好，用TF的InterceptionV3分类

In [17]:
with open("/home/zhoutong/facedata/prepared_data/enthnicity_train_balanced.csv","r") as f:
    content = [i.strip().split("\t") for i in f.readlines()]

In [18]:
set([i[1] for i in content])
for a,b in tqdm_notebook(content[:]):
    p = "/home/zhoutong/facedata/prepared_data/enthnicity_img_copied/{}/{}"
    p = p.format(b,os.path.split(a)[-1])
    img = cv2.imread(a)
    res = cv2.imwrite(p,img)
    if not res:
        print(a,b)
        print(p)

{'ASIAN', 'BLACK', 'INDIA', 'WHITE'}

# Train

In [3]:
import numpy as np
import os
import tensorflow as tf

import mobilenet_v2

import cv2
import math
import json
import pandas as pd
import random
import datetime

In [4]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMAGE_WIDTH = IMG_WIDTH
IMAGE_HEIGHT = IMG_HEIGHT
IMG_CHANNEL = 3
shuffle = True
CHECKPOINTS_DIR = '/home/zhoutong/facedata/prepared_data/ckpt/enthnicity/'

BATCH_SIZE = 20
NUM_EPOCHS = 40000000000

global modelValLoss
global totalValLoss
global valIter
valIter = 0

initialLearningRate = 0.001
learningRateDecay = 0.0000001
weightDecay = 0.0005
label_dim = 4 # {'ASIAN', 'BLACK', 'INDIA', 'WHITE'}
label_dict = {'ASIAN':[1,0,0,0], 'BLACK':[0,1,0,0], 'INDIA':[0,0,1,0], 'WHITE':[0,0,0,1]}

def load_input(fp):
    with open(fp,"r") as f:
        c = [i.strip().split("\t") for i in f.readlines()]
        random.shuffle(c)
        img_fp = [i[0] for i in c]
        label = [label_dict[i[1]] for i in c]
    return img_fp,label

def zprint(inp):
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("|{}| ".format(now)+inp)
    
    
train_img_fp,train_label = load_input('/home/zhoutong/facedata/prepared_data/enthnicity_train_balanced.csv')
test_img_fp,test_label = load_input('/home/zhoutong/facedata/prepared_data/enthnicity_test.csv')

In [5]:
test_label.count(label_dict["ASIAN"])
test_label.count(label_dict["BLACK"])
test_label.count(label_dict["INDIA"])
test_label.count(label_dict["WHITE"])

319

465

287

1929

## TrainEpoch

In [6]:
inputs = tf.placeholder("float32", [None, IMG_HEIGHT, IMG_WIDTH, 3], name="input_to_float")
inptrue = tf.placeholder("float32", [None, 4], name="inptrue")
global_step = tf.placeholder("int32", name="global_step")
logits, pred = mobilenet_v2.mobilenetv2_caffe(inputs, 4) # [0.134,0.34426,0.77456,0.23523], 0.78

In [20]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=inptrue,logits=logits),name='loss')
_ = tf.summary.scalar('loss', loss)
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(inptrue,0), 
                                  predictions=tf.argmax(logits,0))
_ = tf.summary.scalar('acc', acc)
learningRate = tf.train.inverse_time_decay(initialLearningRate, global_step, 1, learningRateDecay, name="ratedecay")
optimizer = tf.train.AdamOptimizer(learningRate, name="adamfull")
train_op = optimizer.minimize(loss, name="trainop")
merged = tf.summary.merge_all()

In [21]:
def train_epoch(train_writer,merged,k,sess):
    i = 0
    global valIter
    batchCounter = 0
    random.shuffle(train_img_fp)
    while i < len(train_img_fp):
        batchTrainImages = []
        batchTrainLabels = []

        j = i
        while j < i + BATCH_SIZE and j < len(train_img_fp):
            label = train_label[j]
            trainImage = cv2.imread(train_img_fp[j])
            if trainImage is None:
                continue

            if trainImage.shape[0] != IMAGE_HEIGHT or trainImage.shape[1] != IMAGE_WIDTH:
                trainImage = cv2.resize(trainImage, (IMAGE_WIDTH, IMAGE_HEIGHT))

            trainImage = trainImage.astype("float32")
            trainImage = trainImage / 255.0
            batchTrainImages.append(trainImage)
            batchTrainLabels.append(label)
            j = j + 1

        acc_,summary,loss, _ = sess.run([acc,merged,"loss:0", "trainop"], feed_dict={
            "input_to_float:0": batchTrainImages,
            "inptrue:0": batchTrainLabels,
            "global_step:0": batchCounter,
        })
        train_writer.add_summary(summary, batchCounter+int(k*(len(train_img_fp)/BATCH_SIZE)))
        
        if batchCounter%400 == 0:
            zprint("[batch]:{} [loss]:{} [acc]:{}".format(batchCounter,loss,acc_))
        batchCounter += 1
        i = i + BATCH_SIZE

In [22]:
def performValidation(sess, k):
    # calculate validation loss first
    i = 0
    batchCounter = 0
    totalValLoss = 0

    global modelValLoss

    while i < len(test_img_fp):
        batchValImages = []
        batchValLabels = []

        j = i
        while j < i + BATCH_SIZE and j < len(test_img_fp):
            label = test_label[j]
            valImage = cv2.imread(test_img_fp[j])
            if valImage is None:
                continue


            if valImage.shape[0] != IMAGE_HEIGHT or valImage.shape[1] != IMAGE_WIDTH:
                valImage = cv2.resize(valImage, (IMAGE_WIDTH, IMAGE_HEIGHT))

            valImage = valImage.astype("float32")
            valImage = valImage / 255.0

            batchValLabels.append(label)
            batchValImages.append(valImage)
            j = j + 1

        acc_,loss = sess.run([acc,"loss:0"], feed_dict={
            "input_to_float:0": batchValImages,
            "inptrue:0": batchValLabels,
            "global_step:0": batchCounter,
        })

        #loss = loss[0] / float(len(batchValImages))

        totalValLoss += loss
        batchCounter += 1
        i = i + BATCH_SIZE

    zprint("[Test-iteration]: {} [loss]: {} [acc]:{}".format(k, totalValLoss / float(batchCounter),acc_))
    if (k == 0):
        modelValLoss = totalValLoss
        saver.save(sess, CHECKPOINTS_DIR + "enetperson-" + str(k) + ".ckpt")
        zprint('-'*15+'Saving the model(initial)'+'-'*15)
    else:
        if (totalValLoss < modelValLoss):
            modelValLoss = totalValLoss
            saver.save(sess, CHECKPOINTS_DIR + "enetperson-" + str(k) + ".ckpt")
            zprint('-'*15+'Saving the model'+'-'*15)

In [23]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [24]:
with tf.Session(config=config) as sess:
    saver = tf.train.Saver(max_to_keep=20)
    train_writer = tf.summary.FileWriter('/home/zhoutong/facedata/prepared_data/summaries',sess.graph)
    _,_=sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

    if not os.path.exists(CHECKPOINTS_DIR):
        os.makedirs(CHECKPOINTS_DIR)
    else:
        ckpt_path = tf.train.latest_checkpoint(CHECKPOINTS_DIR)
        print("ckpt_path: ",ckpt_path)
        if not ckpt_path is None:
            print('restore the last training', ckpt_path)
            saver.restore(sess, ckpt_path)
        else:
            print('no restore this time', ckpt_path)

    print('----------------------begin trainng...---------------------------------------')
    k = 0
    writer = tf.summary.FileWriter(CHECKPOINTS_DIR, sess.graph)
    while k < NUM_EPOCHS:
        # start training
        print( "Starting training")
        train_epoch(train_writer,merged,k,sess)
        performValidation(sess, k)
        print( "Epoch training complete ", k)
        k = k + 1

ckpt_path:  None
no restore this time None
----------------------begin trainng...---------------------------------------
Starting training
|2019-07-12 17:36:53| [batch]:0 [loss]:3.5287766456604004 [acc]:0.0
|2019-07-12 17:37:07| [batch]:400 [loss]:1.4619675874710083 [acc]:0.0
|2019-07-12 17:37:21| [batch]:800 [loss]:1.3940198421478271 [acc]:0.0
|2019-07-12 17:37:34| [batch]:1200 [loss]:1.4577248096466064 [acc]:0.0
|2019-07-12 17:37:48| [batch]:1600 [loss]:1.423396348953247 [acc]:0.0
|2019-07-12 17:38:04| [Test-iteration]: 0 [loss]: 1.378499643802643 [acc]:0.0
|2019-07-12 17:38:09| ---------------Saving the model(initial)---------------
Epoch training complete  0
Starting training
|2019-07-12 17:38:09| [batch]:0 [loss]:1.3989359140396118 [acc]:0.0
|2019-07-12 17:38:23| [batch]:400 [loss]:1.413307547569275 [acc]:0.0
|2019-07-12 17:38:36| [batch]:800 [loss]:1.4192262887954712 [acc]:0.0
|2019-07-12 17:38:50| [batch]:1200 [loss]:1.391453504562378 [acc]:0.0
|2019-07-12 17:39:04| [batch]:1600

|2019-07-12 17:57:25| [Test-iteration]: 16 [loss]: 1.379791255791982 [acc]:0.0
Epoch training complete  16
Starting training
|2019-07-12 17:57:25| [batch]:0 [loss]:1.3955031633377075 [acc]:0.0
|2019-07-12 17:57:39| [batch]:400 [loss]:1.37494695186615 [acc]:0.0
|2019-07-12 17:57:53| [batch]:800 [loss]:1.3988714218139648 [acc]:0.0
|2019-07-12 17:58:07| [batch]:1200 [loss]:1.3931125402450562 [acc]:0.0
|2019-07-12 17:58:21| [batch]:1600 [loss]:1.3864628076553345 [acc]:0.0
|2019-07-12 17:58:37| [Test-iteration]: 17 [loss]: 1.3850239316622417 [acc]:0.0
Epoch training complete  17
Starting training
|2019-07-12 17:58:37| [batch]:0 [loss]:1.3823286294937134 [acc]:0.0
|2019-07-12 17:58:51| [batch]:400 [loss]:1.3803914785385132 [acc]:0.0
|2019-07-12 17:59:05| [batch]:800 [loss]:1.383434534072876 [acc]:0.0
|2019-07-12 17:59:19| [batch]:1200 [loss]:1.393185019493103 [acc]:0.0
|2019-07-12 17:59:33| [batch]:1600 [loss]:1.3882770538330078 [acc]:0.0
|2019-07-12 17:59:49| [Test-iteration]: 18 [loss]: 1.

|2019-07-12 18:18:04| [batch]:400 [loss]:1.3734428882598877 [acc]:0.0
|2019-07-12 18:18:18| [batch]:800 [loss]:1.3763115406036377 [acc]:0.0
|2019-07-12 18:18:32| [batch]:1200 [loss]:1.3911914825439453 [acc]:0.0
|2019-07-12 18:18:46| [batch]:1600 [loss]:1.3851244449615479 [acc]:0.0
|2019-07-12 18:19:02| [Test-iteration]: 34 [loss]: 1.4192441447575888 [acc]:0.0
Epoch training complete  34
Starting training
|2019-07-12 18:19:02| [batch]:0 [loss]:1.3676503896713257 [acc]:0.0
|2019-07-12 18:19:17| [batch]:400 [loss]:1.3767532110214233 [acc]:0.0
|2019-07-12 18:19:31| [batch]:800 [loss]:1.3903661966323853 [acc]:0.0
|2019-07-12 18:19:45| [batch]:1200 [loss]:1.391844391822815 [acc]:0.0
|2019-07-12 18:19:59| [batch]:1600 [loss]:1.3765583038330078 [acc]:0.0
|2019-07-12 18:20:15| [Test-iteration]: 35 [loss]: 1.3789726710319519 [acc]:0.0
Epoch training complete  35
Starting training
|2019-07-12 18:20:15| [batch]:0 [loss]:1.3808479309082031 [acc]:0.0
|2019-07-12 18:20:29| [batch]:400 [loss]:1.378596

|2019-07-12 18:39:02| [batch]:1200 [loss]:1.4029687643051147 [acc]:0.0
|2019-07-12 18:39:16| [batch]:1600 [loss]:1.3860973119735718 [acc]:0.0
|2019-07-12 18:39:32| [Test-iteration]: 51 [loss]: 1.3834144798914592 [acc]:0.0
Epoch training complete  51
Starting training
|2019-07-12 18:39:32| [batch]:0 [loss]:1.3776676654815674 [acc]:0.0
|2019-07-12 18:39:45| [batch]:400 [loss]:1.3807240724563599 [acc]:0.0
|2019-07-12 18:39:59| [batch]:800 [loss]:1.3808238506317139 [acc]:0.0
|2019-07-12 18:40:13| [batch]:1200 [loss]:1.3956058025360107 [acc]:0.0
|2019-07-12 18:40:27| [batch]:1600 [loss]:1.3742315769195557 [acc]:0.0
|2019-07-12 18:40:43| [Test-iteration]: 52 [loss]: 1.3728743116060893 [acc]:0.0
Epoch training complete  52
Starting training
|2019-07-12 18:40:43| [batch]:0 [loss]:1.3891575336456299 [acc]:0.0
|2019-07-12 18:40:57| [batch]:400 [loss]:1.370185136795044 [acc]:0.0
|2019-07-12 18:41:11| [batch]:800 [loss]:1.3842862844467163 [acc]:0.0
|2019-07-12 18:41:25| [batch]:1200 [loss]:1.39914

limit_output extension: Maximum message size of 30000 exceeded with 30038 characters

KeyboardInterrupt: 

# Predict

In [28]:
label_dict_ = {'ASIAN':[1,0,0,0], 'BLACK':[0,1,0,0], 'INDIA':[0,0,1,0], 'WHITE':[0,0,0,1]}
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    _,_=sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    ckpt_path = tf.train.latest_checkpoint(CHECKPOINTS_DIR)
    print("ckpt_path: ",ckpt_path)
    valImage = cv2.imread("/home/zhoutong/facedata/prepared_data/face_img/6573530_054.jpg")
    if valImage.shape[0] != IMAGE_HEIGHT or valImage.shape[1] != IMAGE_WIDTH:
        valImage = cv2.resize(valImage, (IMAGE_WIDTH, IMAGE_HEIGHT))
    valImage = valImage.astype("float32")
    valImage = valImage / 255.0
    l,p = sess.run([logits,pred],feed_dict={inputs:[valImage]})
    l
    p
    l.argmax()
    p.argmax()

ckpt_path:  /home/zhoutong/facedata/prepared_data/ckpt/enthnicity/enetperson-11.ckpt


array([[-1.5938715,  6.6204023,  2.8467655, -2.4067674]], dtype=float32)

array([[2.6458074e-04, 9.7717404e-01, 2.2444105e-02, 1.1736043e-04]],
      dtype=float32)

1

1

# Test

In [115]:
with open("/home/zhoutong/facedata/CASIA-maxpy-clean/0000133/007.json","r") as f:
    json_str = json.load(f)
json_str

{'image_id': 'm4WbxItQOmh4UbEv/OTlJQ==',
 'request_id': '1553479612,348b3e30-4235-441a-b0cb-a32b47ce89e5',
 'time_used': 211,
 'faces': [{'landmark': {'contour_chin': {'y': 217, 'x': 105},
    'left_eye_upper_left_quarter': {'y': 98, 'x': 97},
    'mouth_lower_lip_right_contour1': {'y': 179, 'x': 134},
    'left_eye_bottom': {'y': 106, 'x': 101},
    'mouth_lower_lip_right_contour2': {'y': 182, 'x': 142},
    'contour_left7': {'y': 140, 'x': 71},
    'contour_left6': {'y': 131, 'x': 72},
    'contour_left5': {'y': 123, 'x': 73},
    'contour_left4': {'y': 115, 'x': 75},
    'contour_left3': {'y': 107, 'x': 77},
    'contour_left2': {'y': 99, 'x': 80},
    'contour_left1': {'y': 91, 'x': 82},
    'left_eye_lower_left_quarter': {'y': 103, 'x': 96},
    'contour_right1': {'y': 117, 'x': 208},
    'contour_right3': {'y': 137, 'x': 206},
    'contour_right2': {'y': 127, 'x': 207},
    'contour_right5': {'y': 156, 'x': 201},
    'contour_right4': {'y': 147, 'x': 204},
    'contour_right7': {

In [131]:
import pandas as pd
examples = pd.read_csv("/home/zhoutong/data_train_casia.csv")
examples.head(3)
imageNames = []
for _, row in examples.iterrows():
    linedata = row
    imname, jsonname = linedata['jpg'], linedata['json']
    imageNames.append((imname, jsonname))
imageNames[0]

,jpg,json
0,myCasia/CASIA-maxpy-clean_2471798_154.jpg,myCasia/CASIA-maxpy-clean_2471798_154.json
1,myCasia/CASIA-maxpy-clean_0000981_053.jpg,myCasia/CASIA-maxpy-clean_0000981_053.json
2,myCasia/CASIA-maxpy-clean_0000233_346.jpg,myCasia/CASIA-maxpy-clean_0000233_346.json


('myCasia/CASIA-maxpy-clean_2471798_154.jpg',
 'myCasia/CASIA-maxpy-clean_2471798_154.json')